In [1]:
%config InlineBackend.rc = {"figure.dpi": 72, 'figure.figsize': (6.0, 4.0)}
%matplotlib ipympl

import ase
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

import abtem

abtem.config.set({"local_diagnostics.progress_bar": True});
abtem.config.set({"device": "cpu"});

# Interactive visualizations

## Probe

In [101]:
probe = abtem.Probe(
    semiangle_cutoff=30,
    defocus=np.linspace(-70, 70, 30),
    astigmatism=np.linspace(-70, 70, 30),
    astigmatism_angle=np.pi / 4,
    gpts=256,
    extent=10,
    energy=200e3,
)
probes = probe.build().intensity().compute()  # .compute()

[########################################] | 100% Completed | 658.23 ms


In [103]:
visualization = probes.show(cmap="viridis", display=False, cbar=True)

visualization.widgets

## HRTEM

In [113]:
tube1 = ase.build.nanotube(10, 0, length=5)
tube2 = ase.build.nanotube(16, 0, length=5)
tubes = tube1 + tube2
tubes.center(vacuum=4.0, axis=(0, 1))

rotated_tubes = tubes.copy()
rotated_tubes.rotate("y", 90, rotate_cell=True)
rotated_tubes = abtem.standardize_cell(rotated_tubes)

potential = abtem.Potential(
    rotated_tubes,
    sampling=0.05,
    projection="infinite",
    slice_thickness=1,
    exit_planes=1
)

#abtem.show_atoms(tubes, plane="xy", title="Beam view");

In [114]:
wave = abtem.PlaneWave(energy=100e3)

exit_wave = wave.multislice(potential).intensity().compute()

[########################################] | 100% Completed | 1.09 ss


In [115]:
visualization = (
    exit_wave[:]
    .show(interact=True, axes_types=["index"])
)

## STEM

In [87]:
atoms = ase.build.mx2(vacuum=3)
atoms = abtem.orthogonalize_cell(atoms) * (3, 2, 1)

potential = abtem.Potential(atoms, projection="infinite", slice_thickness=10)#.build().compute()

scan = abtem.GridScan(
    start=(0, 0), end=(1 / 3, 1 / 2), fractional=True, potential=potential,
)

doses = np.geomspace(1e1, 1e7, 10)

measurements = (
    probe.scan(potential, scan=scan)
    .integrate_radial(50, 150)
    #.to_images()
    .gaussian_filter(0.3)
    .interpolate(.1)
    .tile((3, 2))
    .poisson_noise(dose_per_area=doses)
    .compute()
)

[########################################] | 100% Completed | 41.54 s


In [88]:
%matplotlib ipympl

In [90]:
visualization = (
    measurements[:, :, :, :]
    .show(interact=True, axes_types=["index", "index", "index", "index"], cbar=True)
)